In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.cluster import KMeans

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [24]:
trans = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data = torchvision.datasets.ImageFolder(root='Images', transform=trans)
traindata, testvaldata = torch.utils.data.random_split(data, [12000, 8580])
valdata, testdata = torch.utils.data.random_split(testvaldata, [4000, 4580])

trainloader = torch.utils.data.DataLoader(traindata, batch_size=1, shuffle=True)
valloader = torch.utils.data.DataLoader(valdata, batch_size=1, shuffle=True)
testloader = torch.utils.data.DataLoader(testdata, batch_size=1, shuffle=True)

In [25]:
import torchvision.models as models
net = models.resnet50(pretrained=True)
num = net.fc.in_features
net.fc = nn.Linear(num, 120)

PATH = './resnet50.pth'
net.load_state_dict(torch.load(PATH))

layer = net._modules.get('avgpool')
net.eval()
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [26]:
"""   K MEANS   """

features = np.empty((0, 2048))
train_labels = np.zeros(0)
running_loss = 0.0
for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0].to(device), data[1].to(device)
    my_embedding = torch.zeros(2048)
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))

    h = layer.register_forward_hook(copy_data)

    net(inputs)
    h.remove()

    features = np.append(features, my_embedding.reshape(1, len(my_embedding)), axis=0)
    train_labels = np.append(train_labels, labels.cpu().numpy(), axis=0)
print('Finished Training')
print(features.shape)

Finished Training
(12000, 2048)


In [27]:
testfeatures = np.empty((0, 2048))
testlabels = np.zeros(0)
for i, data in enumerate(valloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0].to(device), data[1].to(device)
    my_embedding = torch.zeros(2048)
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))

    h = layer.register_forward_hook(copy_data)
    
    net(inputs)
    h.remove()

    testfeatures = np.append(testfeatures, my_embedding.reshape(1, len(my_embedding)), axis=0)
    testlabels = np.append(testlabels, labels.cpu().numpy(), axis=0)
print(testfeatures.shape)

(4000, 2048)


In [28]:
from sklearn import svm
clf = svm.SVC()
clf.fit(features, train_labels)
preds = clf.predict(testfeatures)

In [29]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(testlabels, preds)
np.savetxt('svm_cnn_confusion.txt', cm)

from sklearn.metrics import accuracy_score
meanacc = accuracy_score(testlabels, preds)*100
classacc = 100*(cm.diagonal()/cm.sum(axis=1))
f = open("classes.txt", "r")
classmap = {}
for i in range(120):
    classs = f.readline().split(" ")
    classmap[classs[0]] = classacc[i]
    
sortedmap = sorted(classmap.items(), key=lambda item: item[1], reverse=True)

for k, v in sortedmap:
    print('Accuracy of %30s : %10d %%' % (k, v))
print('Mean accuracy: %5f %%' % meanacc)

Accuracy of               Japanese_spaniel :        100 %
Accuracy of                    toy_terrier :        100 %
Accuracy of                   Afghan_hound :        100 %
Accuracy of                         basset :        100 %
Accuracy of                     bloodhound :        100 %
Accuracy of             Norwegian_elkhound :        100 %
Accuracy of             Bedlington_terrier :        100 %
Accuracy of                 Border_terrier :        100 %
Accuracy of                Norwich_terrier :        100 %
Accuracy of                       Airedale :        100 %
Accuracy of                 Scotch_terrier :        100 %
Accuracy of    West_Highland_white_terrier :        100 %
Accuracy of          flat_coated_retriever :        100 %
Accuracy of         curly_coated_retriever :        100 %
Accuracy of    German_short_haired_pointer :        100 %
Accuracy of                 English_setter :        100 %
Accuracy of                   Irish_setter :        100 %
Accuracy of   